In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as scipy
import matplotlib.pyplot as plt
import requests
import io 
from __future__ import division
import seaborn as sns
%matplotlib inline

url = "http://www.football-data.co.uk/mmz4281/1718/E0.csv"
testfile = requests.get(url).content
df=pd.read_csv(io.StringIO(testfile.decode('utf-8')))
df.to_csv("./E0_17.csv")
df = pd.read_csv("./E0_17.csv")
df.columns
df

#data cleaning
res_16 = df.iloc[:,:23]
res_16 = res_16.drop(['Div','Date'],axis=1)
table_features = df.iloc[:,:7]
table_features = table_features.drop(['FTHG','FTAG','Div','Date'],axis=1)
bet_16 = df.iloc[:,23:]
res_16.head()
feature_table = df.iloc[:,:23]
res_16.tail()

table_16 = pd.DataFrame(columns=('Team','HGS','AGS','HAS','AAS','HGC','AGC','HDS','ADS',))
table_16 = table_16[:-10]

res_16 = res_16[:-10]
res_16.shape[0]

avg_home_scored_16 = res_16.FTHG.sum()*1.0 / res_16.shape[0]
avg_away_scored_16 = res_16.FTAG.sum()*1.0 / res_16.shape[0]
avg_home_conceded_16 = avg_away_scored_16
avg_away_conceded_16 = avg_home_scored_16
print("Average number of goals at home",avg_home_scored_16)
print("Average number of goals away", avg_away_scored_16)
print("Average number of goals conceded at home",avg_home_conceded_16)
print("Average number of goals conceded away",avg_away_conceded_16)

res_home = res_16.groupby('HomeTeam')
# res_home.HomeTeam.all()
res_away = res_16.groupby('AwayTeam')
# res_away.AwayTeam.any()

table_16.Team = res_home.HomeTeam.first().values

table_16.HGS = res_home.FTHG.sum().values
table_16.HGC = res_home.FTAG.sum().values
table_16.AGS = res_away.FTAG.sum().values
table_16.AGC = res_away.FTHG.sum().values

num_games = res_16.shape[0]/20

table_16.HAS = (table_16.HGS / num_games) / avg_home_scored_16
table_16.AAS = (table_16.AGS / num_games) / avg_away_scored_16
table_16.HDS = (table_16.HGC / num_games) / avg_home_conceded_16
table_16.ADS = (table_16.AGC / num_games) / avg_away_conceded_16
table_16

#feature_table contains all the fixtures in the current season.
feature_table = feature_table[['HomeTeam','AwayTeam','FTR','HST','AST']]
f_HAS = []
f_HDS = []
f_AAS = []
f_ADS = []
for index,row in feature_table.iterrows():
    f_HAS.append(table_16[table_16['Team'] == row['HomeTeam']]['HAS'].values[0])
    f_HDS.append(table_16[table_16['Team'] == row['HomeTeam']]['HDS'].values[0])
    f_AAS.append(table_16[table_16['Team'] == row['AwayTeam']]['AAS'].values[0])
    f_ADS.append(table_16[table_16['Team'] == row['AwayTeam']]['ADS'].values[0])
    
feature_table['HAS'] = f_HAS
feature_table['HDS'] = f_HDS
feature_table['AAS'] = f_AAS
feature_table['ADS'] = f_ADS

feature_table.head(20)

def transformResult(row):
    '''Converts results (H,A or D) into numeric values'''
    if(row.FTR == 'H'):
        return 1
    elif(row.FTR == 'A'):
        return 0
    else:
        return 2

feature_table["Result"] = feature_table.apply(lambda row: transformResult(row),axis=1)
feature_table = feature_table[:-10] #ONLY FOR WEEK 33. REMOVE FROM 34
feature_table.tail(10)

X_train = feature_table[['HST','AST','HAS','HDS','AAS','ADS',]]
y_train = feature_table['Result']

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import voting_classifier
from sklearn.svm import LinearSVC

clf1 = RandomForestClassifier()
clf2 = MultinomialNB()
clf3 = XGBClassifier()
clf4 = LinearSVC()

ht = res_16.loc[40].HomeTeam
at = res_16.loc[40].AwayTeam
feat_table = res_16.sort_index(ascending=False)
feat_table = feat_table[['HomeTeam','AwayTeam','FTR','FTHG','FTAG','HS','AS','HC','AC']]
# Adding next week fixtures
new_fixtures = pd.DataFrame( [['Leicester','Man United','D',0,0,0,0,0,0],
                             ['Burnley','Tottenham','D',0,0,0,0,0,0],
                             ['Swansea','Crystal Palace','D',0,0,0,0,0,0],
                             ['West Ham','Newcastle','D',0,0,0,0,0,0],
                             ['Stoke','West Brom','D',0,0,0,0,0,0],
                             ['Southampton','Huddersfield','D',0,0,0,0,0,0],
                             ['Man City','Bournemouth','D',0,0,0,0,0,0],
                             ['Brighton','Watford','D',0,0,0,0,0,0],
                             ['Everton','Chelsea','D',0,0,0,0,0,0],
                             ['Arsenal','Liverpool','D',0,0,0,0,0,0]],columns=feat_table.columns)

new_feat_table = new_fixtures.append(feat_table,ignore_index=True)
new_feat_table = new_feat_table.sort_index(ascending=False)
new_feat_table = new_feat_table.reset_index().drop(['index'], axis=1)
new_feat_table = new_feat_table.sort_index(ascending=False)
feat_table = new_feat_table
feat_table.shape
 
#Adding k recent performance measures
feat_table["pastHS"] = 0.0
feat_table["pastHC"] = 0.0
feat_table["pastAS"] = 0.0
feat_table["pastAC"] = 0.0
feat_table["pastHG"] = 0.0
feat_table["pastAG"] = 0.0

feat_table.head(12)

k = 3

f_HAS = []
f_HDS = []
f_AAS = []
f_ADS = []
for index,row in feat_table.iterrows():
    #print row
    f_HAS.append(table_16[table_16['Team'] == row['HomeTeam']]['HAS'].values[0])
    f_HDS.append(table_16[table_16['Team'] == row['HomeTeam']]['HDS'].values[0])
    f_AAS.append(table_16[table_16['Team'] == row['HomeTeam']]['AAS'].values[0])
    f_ADS.append(table_16[table_16['Team'] == row['HomeTeam']]['ADS'].values[0])
    
feat_table['HAS'] = f_HAS
feat_table['HDS'] = f_HDS
feat_table['AAS'] = f_AAS
feat_table['ADS'] = f_ADS

test_table = feat_table.drop(['FTHG','FTAG','HS','AS','HC','AC'],axis=1)
test_table["Result"] = test_table.apply(lambda row: transformResult(row),axis=1)
test_table.sort_index(inplace=True)
num_games = feat_table.shape[0]-10
num_games

X_train = test_table[['pastHS','pastHC','pastAS','pastAC','pastHG','pastAG','HAS','HDS','AAS','ADS']].loc[0:num_games]
y_train = test_table['Result'].loc[0:num_games]
X_test = test_table[['pastHS','pastHC','pastAS','pastAC','pastHG','pastAG','HAS','HDS','AAS','ADS']].loc[num_games:]
y_test = test_table['Result'].loc[num_games:]

#Adding home advantage

test_table["pastCornerDiff"] = (test_table["pastHC"] - test_table["pastAC"])/k
test_table["pastGoalDiff"] = (test_table["pastHG"] - test_table["pastAG"])/k
test_table["pastShotsDiff"] = (test_table["pastHS"] - test_table["pastAG"])/k

num_games = feat_table.shape[0]-10
v_split = 15
n_games = num_games - v_split

test_table = test_table.fillna(0)
test_table.head()

#                            TRAIN/ TEST

X_train = test_table[['pastCornerDiff','pastGoalDiff','pastShotsDiff','HAS','HDS','AAS','ADS']].loc[0:n_games]
y_train = test_table['Result'].loc[0:n_games]
X_test = test_table[['pastCornerDiff','pastGoalDiff','pastShotsDiff','HAS','HDS','AAS','ADS']].loc[n_games:num_games-1]
y_test = test_table['Result'].loc[n_games:num_games-1]
X_predict = test_table[['pastCornerDiff','pastGoalDiff','pastShotsDiff','HAS','HDS','AAS','ADS']].loc[num_games:]
# X_predict = test_table[['pastCornerDiff','pastGoalDiff','pastShotsDiff','HAS','HDS','AAS','ADS']].loc[:]
X_predict

# ////////////////////////////////////////////////////////////////////////////////////////////////////
#                               KERAS 

# Import necessary modules
import keras
from keras.layers import Dense
from keras.models import Sequential
# from keras.utils import np_utils
from keras.utils import np_utils

# Convert the target to categorical: target
target = np_utils.to_categorical(y_train)

# print("target to_categorical = ",target)
# print("y_train = ",y_train[100:130])

# # Set up the model
model = Sequential()

# # Add hidden layers
model.add(Dense(512, activation='relu', input_shape=(7,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))

# # Add the output layer
model.add(Dense(3, activation='softmax'))

# # # Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Fit the model
model.fit(X_train,target, epochs=10, batch_size=10)

# evaluate the keras model
_, accuracy = model.evaluate(X_train, target)
print('Accuracy of KERAS: %.2f' % (accuracy*100))

# # make prediction
predictions = model.predict(X_test)

# ////////////////////////////////////////////////////////////////////////////////////////////////////
#                          ML algorithms

# #KNN
plot_scores_knn = []
for b in range(1,50):
    clf_knn = KNeighborsClassifier(n_neighbors=b)
    clf_knn.fit(X_train,y_train)
    scores = accuracy_score(y_test,clf_knn.predict(X_test))
    plot_scores_knn.append(scores)

#XGBClassifier
plot_scores_XGB = []
for i in range(1,100):
    clf_XGB = XGBClassifier(n_estimators=i,max_depth=100)
    clf_XGB.fit(X_train, y_train)
    scores = accuracy_score(y_test,clf_XGB.predict(X_test))
    plot_scores_XGB.append(scores)
    
#Logistic Regression
plot_scores_logreg= []
cs = [0.01,0.02,0.1,0.5,1,3,4,5,10]
for c in cs:
    clf_logreg = LogisticRegression(C=c,solver='lbfgs',multi_class='ovr')
    clf_logreg.fit(X_train, y_train)
    scores = accuracy_score(y_test,clf_logreg.predict(X_test))
    plot_scores_logreg.append(scores)

max_knn_n = max(plot_scores_knn)
max_knn_ind = plot_scores_knn.index(max_knn_n)

max_XGB_e = max(plot_scores_XGB)
max_XGB_ind = plot_scores_XGB.index(max_XGB_e) if plot_scores_XGB.index(max_XGB_e)!=0 else 1

max_logreg_c = max(plot_scores_logreg)
max_logreg_ind = plot_scores_logreg.index(max_logreg_c)

clf_knn = KNeighborsClassifier(n_neighbors=max_knn_ind).fit(X_train,y_train)
clf_XGB = XGBClassifier(n_estimators=max_XGB_ind).fit(X_train,y_train)
clf_logreg = LogisticRegression(C=max_logreg_ind,solver='lbfgs',multi_class='ovr').fit(X_train,y_train)

y_pred_knn = clf_knn.predict(X_predict)
y_pred_XGB = clf_XGB.predict(X_predict)
y_pred_logreg = clf_logreg.predict(X_predict)

NN_predict = model.predict(X_predict)

def intToLetterValue(value):
  if(value == 0):
      return 'Loss'
  elif(value == 1):
      return 'Win'
  else:
      return 'Draw'

def map_NN_Results(values):
  mappedResults = []
  for entry in values:
    index = np.argmax(entry)
    letterValue = intToLetterValue(index)
    mappedResults.append(letterValue)
  return mappedResults

def map_ML_Results(values):
  mappedResults = []
  for entry in values:
    letterValue = intToLetterValue(entry)
    mappedResults.append(letterValue)
  return mappedResults

mappedResultsNew = map_NN_Results(NN_predict)
# print(mappedResultsNew)

this_week = test_table[['HomeTeam','AwayTeam']].loc[num_games:]
# this_week = test_table[['HomeTeam','AwayTeam']].loc[:]

this_week['Result_knn'] = map_ML_Results(y_pred_knn)
this_week['Result_XGB'] = map_ML_Results(y_pred_XGB)
this_week['Result_logreg'] = map_ML_Results(y_pred_logreg)
this_week['Result_KERAS']= mappedResultsNew
this_week

print("Accuracy of KNN, XGB, LOG_REG", max_knn_n, max_XGB_e, max_logreg_c)
this_week

Average number of goals at home 1.518918918918919
Average number of goals away 1.1486486486486487
Average number of goals conceded at home 1.1486486486486487
Average number of goals conceded away 1.518918918918919
Epoch 1/10
36/36 [==============================] - 1s 4ms/step - loss: 1.0681 - accuracy: 0.3651
Epoch 2/10
36/36 [==============================] - 0s 4ms/step - loss: 1.0379 - accuracy: 0.4553
Epoch 3/10
36/36 [==============================] - 0s 3ms/step - loss: 1.0095 - accuracy: 0.4773
Epoch 4/10
36/36 [==============================] - 0s 4ms/step - loss: 1.0428 - accuracy: 0.4350
Epoch 5/10
36/36 [==============================] - 0s 4ms/step - loss: 0.9644 - accuracy: 0.5217
Epoch 6/10
36/36 [==============================] - 0s 4ms/step - loss: 1.0223 - accuracy: 0.4667
Epoch 7/10
36/36 [==============================] - 0s 3ms/step - loss: 0.9925 - accuracy: 0.4552
Epoch 8/10
36/36 [==============================] - 0s 3ms/step - loss: 1.0070 - accuracy: 0.4884
Ep

,HomeTeam,AwayTeam,Result_knn,Result_XGB,Result_logreg,Result_KERAS
370,Arsenal,Liverpool,Win,Win,Win,Win
371,Everton,Chelsea,Win,Win,Win,Win
372,Brighton,Watford,Draw,Draw,Win,Win
373,Man City,Bournemouth,Win,Win,Win,Win
374,Southampton,Huddersfield,Loss,Loss,Loss,Win
375,Stoke,West Brom,Loss,Loss,Loss,Loss
376,West Ham,Newcastle,Loss,Loss,Loss,Win
377,Swansea,Crystal Palace,Loss,Loss,Loss,Loss
378,Burnley,Tottenham,Loss,Win,Win,Win
379,Leicester,Man United,Loss,Win,Win,Win
